In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from ripser import ripser
from persim import plot_diagrams
from sklearn.cluster import KMeans
import tadasets
from itertools import combinations
%matplotlib inline

In [12]:
X = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "01", "03", "13",
                 "013", "15", "35", "135", "12", "25", "125", "26", "56", "256", 
                 "02", "06", "026", "36", "036", "34", "45", "345", "57", "47", 
                 "457", "67", "567", "68", "78", "678", "38", "368", "48", "348", "04", "07", "047",
                 "17", "017", "18", "178", "28", "128", "24", "248", "024"] 

In [14]:
def get_faces(lst):
    return [lst[:i] + lst[i+1:] for i in range(len(lst))]


def get_coeff(simplex, faces):
    if simplex in faces:
        idx = faces.index(simplex)
        return 1 if idx%2==0 else -1
    else:
        return 0


def boundary(complex):
    maxdim = len(max(complex, key=len))
    simplices = [sorted([spx for spx in complex if len(spx)==i]) for i in range(1,maxdim+1)]
    bnd = []
    for spx_k, spx_kp1 in zip(simplices, simplices[1:]):
        mtx = []
        for sigma in spx_kp1:
            faces = get_faces(sigma)
            mtx.append([get_coeff(spx, faces) for spx in spx_k])
        bnd.append(np.array(mtx).T)

    return bnd

In [18]:
bnd_X= boundary(X)
bnd_1 =bnd_X[0]

bnd_2 =bnd_X[1]

np.set_printoptions(linewidth=110)
print(bnd_1.T)
print(len(bnd_1))
print(bnd_2.T)
print(len(bnd_2))


[[-1  1  0  0  0  0  0  0  0]
 [-1  0  1  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0]
 [-1  0  0  0  0  0  1  0  0]
 [-1  0  0  0  0  0  0  1  0]
 [ 0 -1  1  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0  0]
 [ 0 -1  0  0  0  1  0  0  0]
 [ 0 -1  0  0  0  0  0  1  0]
 [ 0 -1  0  0  0  0  0  0  1]
 [ 0  0 -1  0  1  0  0  0  0]
 [ 0  0 -1  0  0  1  0  0  0]
 [ 0  0 -1  0  0  0  1  0  0]
 [ 0  0 -1  0  0  0  0  0  1]
 [ 0  0  0 -1  1  0  0  0  0]
 [ 0  0  0 -1  0  1  0  0  0]
 [ 0  0  0 -1  0  0  1  0  0]
 [ 0  0  0 -1  0  0  0  0  1]
 [ 0  0  0  0 -1  1  0  0  0]
 [ 0  0  0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  0  0  1]
 [ 0  0  0  0  0 -1  1  0  0]
 [ 0  0  0  0  0 -1  0  1  0]
 [ 0  0  0  0  0  0 -1  1  0]
 [ 0  0  0  0  0  0 -1  0  1]
 [ 0  0  0  0  0  0  0 -1  1]]
9
[[ 1  0 -1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0 -1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0 -1  0  0

In [16]:
def col_reduce(matrix, c1, c2, row_entry, pivot_rows, pivot_entries):
    scale = (matrix[row_entry][c2]//matrix[row_entry][c1])* -(matrix[row_entry][c1]//matrix[row_entry][c1])
    new_row, new_entry = float("inf"), 0
    for i in range(len(matrix)):
        matrix[i][c2] += scale * matrix[i][c1]
        if matrix[i][c2] != 0:
            new_row = i
            new_entry = matrix[i][c2]
    pivot_rows[c2] = new_row
    pivot_entries[c2]  = new_entry



def red_al(matrix): #caled Kim Funct before
    pivot_rows = []
    pivot_entries =[]
    pivot = False
    for j in range(len(matrix[0])):
        for i in reversed(range(len(matrix))):
            if matrix[i][j] != 0:
                pivot_rows.append(i)
                pivot_entries.append(matrix[i][j])
                pivot = True
                break
        if not pivot:
            pivot_rows.append(float("inf"))
            pivot_entries.append(0)
            pivot = False
            
    for i in range(1, len(pivot_rows)):
        while pivot_rows[i] in pivot_rows[0:i] and pivot_rows[i] != float("inf"):
            col = pivot_rows[0:i].index(pivot_rows[i])
            col_reduce(matrix, col, i, pivot_rows[i], pivot_rows, pivot_entries)
    return matrix

In [17]:
print(re_al(bnd_1))

print(re_al(bnd_2))

[[-1 -1 -1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
[[ 1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 -1 -

In [ ]:
def col_reduce(matrix, c1, c2, row_entry, pivot_rows, pivot_entries, V): #kim function 
    scale = (matrix[row_entry][c2] // matrix[row_entry][c1]) * -(matrix[row_entry][c1] // matrix[row_entry][c1])
    new_row, new_entry = float("inf"), 0
    for i in range(len(matrix)):
        matrix[i][c2] += scale * matrix[i][c1]
        if matrix[i][c2] != 0:
            new_row = i
            new_entry = matrix[i][c2]
    pivot_rows[c2] = new_row
    pivot_entries[c2] = new_entry
    
    # Apply the same operations to V
    V[i][c2] += scale * V[i][c1]
    return V


def reduc_alg(matrix):
    V = np.eye(len(matrix))
    pivot_rows = []
    pivot_entries = []
    pivot = False
    for j in range(len(matrix[0])):
        for i in reversed(range(len(matrix))):
            if matrix[i][j] != 0:
                pivot_rows.append(i)
                pivot_entries.append(matrix[i][j])
                pivot = True
                break
        if not pivot:
            pivot_rows.append(float("inf"))
            pivot_entries.append(0)
            pivot = False

    for i in range(1, len(pivot_rows)):
        while pivot_rows[i] in pivot_rows[0:i] and pivot_rows[i] != float("inf"):
            col = pivot_rows[0:i].index(pivot_rows[i])
            col_reduce(matrix, col, i, pivot_rows[i], pivot_rows, pivot_entries, V)
    return matrix, V

In [21]:
def reductionAlgorithm(R):  #martin function
    numRows = len(R[0])
    V = np.eye(numRows)
    pivotRows, pivotEntries = [[] for i in range (numRows)], [[] for i in range (numRows)]

    for i in range(numRows):
        h = 0
        while (h == 0):
            pivotRow = np.inf
            for j in range(numRows - 1, -1, -1):
                if R[j][i] != 0:
                    pivotRow = j
                    break;
            if pivotRow == np.inf:
                pivotEntries[i] = 0
            else:
                pivotEntries[i] = R[pivotRow][i]
            pivotRows[i] = pivotRow
            if pivotRow == np.inf or all(pivotRows[k] != pivotRows[i] for k in range(i)):
                h = 1
                break;
            for k in range(i):
                if pivotRows[k] == pivotRows[i]:
                    c = pivotEntries[i] // pivotEntries[k]
                    for l in range(numRows):
                        R[l][i] -= c * R[l][k]
                        V[l][i] -= c * V[l][k]            
    return R, V

reductionAlgorithm(bnd_2)

(array([[ 1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  1,  0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0],
        [-1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0],
        [ 0,  0, -1,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  0, -1,  0,  1,  0,  0,  0,  0],
        [ 0, -1,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  1, -1,  0,  0,  0,  0,  0,  0],
        [ 1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,  1, -1,  0,  0,  0,  0,  0],
        [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,